In [1]:
from math import sin, atan, sqrt
from ipywidgets import interact, interactive, FloatSlider, IntSlider, Checkbox, Layout, HBox, VBox
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# The pacejka magic formula
def pacejkaMF(slip, B, C, D, E):
    '''
    slip = slip ratio (from -100 - 100), or angle (-90 to 90)

    B = define the width of the peak + how sharp it is

    C = define the height of the peak

    D = vertical scale of the shape
    
    E = how sharp it drops after peak
    '''
    Bx = B * slip
    return D * sin( C * atan( Bx - E * ( Bx - atan(Bx) ) ) )

def findMaxSlip(B,C,D,E):
    f = 0.0
    step = 0.0001
    slip = 0.0
    while (slip <= 100.0):
        newF = pacejkaMF(slip, B, C, D, E)
        if newF >= f:
#             print(f"still climbing: {newF:.4f}")
            f = newF
        else:
#             print(f"max slip @ {slip:.2f} -> {newF:.4f} < {f:.4f}")
            return slip
        slip += step
    return slip

def normalized(x, y):
    s = sqrt(x*x + y*y)
    if s <= 0.0:
        return (x, y)
    return (x/s, y/s)

In [3]:
slip_ratio = np.linspace(-100.0, 100.0, 200)
slip_angle = np.linspace(-90.0, 90.0, 200)

def plotF(Blon, Clon, Dlon, Elon, Blat, Clat, Dlat, Elat, showMax = True, slipRange=1.5, gridSize=50):
    fn = np.vectorize(pacejkaMF)
    Flon = fn(slip_ratio, Blon, Clon, Dlon, Elon)
    Flat = fn(slip_angle, Blat, Clat, Dlat, Elat)
    
    # find max slip
    max_slip = findMaxSlip(Blon, Clon, Dlon, Elon)
    max_angle = findMaxSlip(Blat, Clat, Dlat, Elat)
    
    # fig, axs = plt.subplots(2, 1, figsize=(10, 10), gridspec_kw={'height_ratios': [1, 2]})
    fig = plt.figure(figsize=(10, 10))
    gs = fig.add_gridspec(4, 1)
    fig.canvas.header_visible = False

    p1 = fig.add_subplot(gs[0, :])
    p2 = fig.add_subplot(gs[1:, :], projection='3d')

    # 2D Lateral + Longitudinal Tire Forces Stacked
    p1.plot(slip_ratio, Flon, color='g', label = 'Flon')
    p1.plot(slip_angle, Flat, color='orange', label = 'Flat')
    p1.axvline(0.0, color='k')

    if showMax:
        p1.axvline(max_slip, color='r', linestyle='dashed', label = 'max_slip')
        p1.text(max_slip, 0.0, "max_slip = %.4f" % (max_slip))

        p1.axvline(max_angle, color='brown', linestyle='dashed', label = 'max_angle')
        p1.text(max_angle, 0.5, "max_angle = %.4f" % (max_angle))
    p1.grid()
    p1.legend(loc='lower right')

    # Combined Grip (Traction circle)
    ratio = np.linspace(-slipRange, slipRange, gridSize)
    X, Y = np.meshgrid(ratio, ratio)
    # X2, Y2 = np.meshgrid(ratio * max_slip, ratio * max_angle)

    # use x, y as scaler, x2, y2 as parameter to compute real forces
    s = np.sqrt(X ** 2 + Y ** 2)
    sx, sy = np.vectorize(normalized)(X, Y)
    
    Fx = sx * fn(s * max_slip, Blon, Clon, Dlon, Elon)
    Fy = sy * fn(s * max_angle, Blat, Clat, Dlat, Elat)

    Fc = np.sqrt(Fx ** 2 + Fy ** 2)

    surf = p2.plot_surface(X, Y, Fc, cmap='coolwarm', edgecolor='k', linewidth=0.25, alpha=0.7)
    p2.set_xlabel('slip_ratio / optimal_slip')
    p2.set_ylabel('slip_angle / optimal_angle')
    cb = fig.colorbar(surf, shrink=0.5, aspect=16)
    cb.solids.set_edgecolor('face')

    plt.tight_layout(h_pad = 0, w_pad = 0)
    
    plt.show()

fw = Layout(width='80%')

Blon=FloatSlider(min=0.0, max=2.0, step=0.01, value=0.82, layout = fw)
Clon=FloatSlider(min=0.0, max=2.0, step=0.01, value=1.55, layout = fw)
Dlon=FloatSlider(min=0.0, max=1.0, step=0.01, value=1.0, layout = fw)
Elon=FloatSlider(min=-1.0, max=1.0, step=0.01, value=-0.4, layout = fw)

Blat=FloatSlider(min=0.0, max=2.0, step=0.01, value=0.714, layout = fw)
Clat=FloatSlider(min=0.0, max=2.0, step=0.01, value=1.5, layout = fw)
Dlat=FloatSlider(min=0.0, max=1.0, step=0.01, value=1.0, layout = fw)
Elat=FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.2, layout = fw)

showMax = Checkbox(value=True)
slipRange = FloatSlider(min=1.0, max=10.0, step=0.01, value=1.5, description='slip range(% of optima)')
gridSize = IntSlider(min=10, max=200, value=50, description='grid size')

# plotF(0.514, 1.45, 1.0, -0.2)
quarter_w = Layout(width='25%')
iplot = interactive(plotF,
    Blon = Blon,
    Clon = Clon,
    Dlon = Dlon,
    Elon = Elon,
    Blat = Blat,
    Clat = Clat,
    Dlat = Dlat,
    Elat = Elat,
    showMax = showMax,
    slipRange = slipRange,
    gridSize = gridSize
)

l_50 = Layout(display='flex', width='50%', border='solid', align_items='center')
l_100 = Layout(display='flex', width='100%', border='solid', align_items='center', flex_flow='column')

left_controls = VBox([Blon, Clon, Dlon, Elon], layout = l_50)
right_controls = VBox([Blat, Clat, Dlat, Elat], layout = l_50)
top_controls = HBox([left_controls, right_controls])
bottom_controls = HBox([showMax, slipRange, gridSize])
controls = VBox([top_controls, bottom_controls])
output = HBox([iplot.children[-1]], layout = l_100)

display(VBox([controls, output]))
iplot.update()